In [2]:
#dependencies
import json
import csv
import os
import time
from datetime import timedelta, date, datetime
import requests
import pandas as pd
import numpy as np
from pprint import pprint
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.gridspec import GridSpec

In [3]:
#transit_df
airport_data = pd.read_csv("data.csv")
airport_data.head()

FileNotFoundError: File b'data.csv' does not exist

In [ ]:
airport_data["FLIGHTS"].unique()

In [ ]:
#import airline names and convert to dict

carrier_names = pd.read_csv("L_UNIQUE_CARRIERS.csv")

carrier_names = carrier_names.rename(columns={"Description":"Airline", "Code":"CARRIER"})

abbr_carrier = carrier_names.set_index("CARRIER")["Airline"].to_dict()

In [ ]:
#find top ten arirports

top_ten_dest = airport_data["FLIGHTS"].groupby(airport_data["DEST"]).sum().reset_index()
top_ten_dest = top_ten_dest.sort_values(by="FLIGHTS", ascending=False)
top_ten_dest = top_ten_dest.head(10)

top_ten_orig = airport_data["FLIGHTS"].groupby(airport_data["ORIGIN"]).sum().reset_index()
top_ten_orig = top_ten_orig.sort_values(by="FLIGHTS", ascending=False)
top_ten_orig = top_ten_orig.head(10)

top_ten_orig.to_csv("Top Ten Origins")
top_ten_dest.to_csv("Top Ten Destinations")
top_ten_orig

In [ ]:
#delays by airline
airline_delays = airport_data.loc[:,["CARRIER", "Total_Delay", "DEP_DELAY", "ARR_DELAY"]]

In [ ]:
#basic descriptive stats for airlines

def get_stats(group):
    return {"Min Delay" : group.min(), "Max Delay" : group.max(), "Number Flight Delays" : group.count(),
            "Mean Departure Delay" : group.mean()}

stat_overview = airport_data
stat_overview["CARRIER"] = stat_overview["CARRIER"].replace(abbr_carrier)
stat_overview = airport_data["DEP_DELAY"].groupby(airport_data["CARRIER"]).apply(get_stats).unstack()
stat_overview = stat_overview.sort_values("Number Flight Delays")
stat_overview = stat_overview[["Max Delay", "Min Delay", "Mean Departure Delay", "Number Flight Delays"]]
stat_overview = stat_overview.sort_values("Mean Departure Delay")
stat_overview

In [ ]:
#get number of flights for "worst" airline by departure airport

jet_blue = airport_data.loc[airport_data["CARRIER"] == "JetBlue Airways"]
jb_orig = pd.DataFrame(jet_blue.groupby("ORIGIN").FLIGHTS.count())
jb_orig = jb_orig.sort_values("FLIGHTS", ascending=False)
jb_orig.head(10)

In [ ]:
#get number of flights for "best" airline by departure airport

ha = airport_data.loc[airport_data["CARRIER"] == "Hawaiian Airlines Inc."]
ha_orig = pd.DataFrame(ha.groupby("ORIGIN").FLIGHTS.count())
ha_orig = ha_orig.sort_values("FLIGHTS", ascending=False)
ha_orig.head(10)

In [ ]:
#get df with only top ten airports as origin
top_ten = airport_data[airport_data["ORIGIN"].isin(top_ten_orig["ORIGIN"])]

In [ ]:
#calc mean dep delay and sd by origin
top_ten_stats = []
airline_stats = []

for origin in top_ten["ORIGIN"].unique():
    airport_stats = {}
    temp = top_ten.loc[top_ten["ORIGIN"] == origin]
    airport_stats["Airport"] = origin
    airport_stats["Mean"] = temp["DEP_DELAY"].mean()
    airport_stats["STD"] = np.std(temp["DEP_DELAY"])
    top_ten_stats.append(airport_stats)

    #calc stats for each airline
    for carrier in airport_data["CARRIER"].unique():
        carrier_stats = {}
        temp2 = temp.loc[temp["CARRIER"] == carrier]
        carrier_stats["Airline"] = carrier
        carrier_stats["Airport"] = origin
        carrier_stats["zscore"] = (temp2["DEP_DELAY"].mean() - temp["DEP_DELAY"].mean()) / np.std(temp["DEP_DELAY"])
        airline_stats.append(carrier_stats)
        
top_ten_stats = pd.DataFrame(top_ten_stats)
airline_stats = pd.DataFrame(airline_stats)
airline_stats = airline_stats.fillna(0)
#airline_stats

In [ ]:
#get mean zscore for each ailrine grouped by airoprt and sorted

airline_comparison = airline_stats.groupby("Airline").mean()
airline_comparison = airline_comparison.sort_values("zscore")
airline_comparison

In [ ]:
#calc avg z score by airline

plt.figure(figsize=(20,10))

x_axis = np.arange(0,len(airline_comparison.index))
tick_locations = []
for x in x_axis:
    tick_locations.append(x)


plt.xlabel("Airline Relative Performance at Busiest Airports", fontsize=18, bbox={"facecolor":"midnightblue", "pad":5}, color='w', labelpad=20)
plt.ylabel("Mean Z-Score", fontsize=14)
plt.xticks(rotation=45)

plt.bar(x_axis, airline_comparison["zscore"], facecolor="firebrick", alpha=0.75, align="edge",)
plt.xticks(tick_locations, airline_comparison.index)
plt.show()


In [ ]:
#create df with airlines and delays and replace code with airline name
airline_delays["CARRIER"] = airline_delays["CARRIER"].replace(abbr_carrier)

fig = plt.figure(1, figsize=(16,15))
gs=GridSpec(2,2)
ax=fig.add_subplot(gs[0,:])

colors = ['firebrick', 'gold', 'lightcoral', 'aquamarine', 'yellowgreen', 
          'seagreen', 'tomato', 'violet', 'wheat', 'chartreuse', 'lightskyblue', 'royalblue']

ax = sns.stripplot(y="CARRIER", x="DEP_DELAY", size=6, palette=colors, data=airline_delays, linewidth=.5, jitter=True)
plt.setp(ax.get_xticklabels(), fontsize=14)
plt.setp(ax.get_yticklabels(), fontsize=14)
ax.set_xticklabels(['{:2.0f}h{:2.0f}m'.format(*[int(y) for y in divmod(x,60)])
                         for x in ax.get_xticks()])
ax.yaxis.label.set_visible(False)
plt.xlabel("Departure Delay in Minutes by Airline", fontsize=18, bbox={"facecolor":"midnightblue", "pad":5}, color='w', labelpad=20)
#plt.figure(figsize=(20,10))
plt.savefig('delay_in_min_by_airline.png')
plt.show()


DENNIS' CODE

In [ ]:
sns.set_palette("deep", desat=.6)
colors = sns.color_palette("deep")
sns.set_context(rc={"figure.figsize": (50, 20)})
pd.set_option('display.max_columns', 500)
transit_csv= "data.csv"
transit_df = pd.read_csv(transit_csv)

In [ ]:
transit_df.count()

In [ ]:
transit_df.fillna(0, inplace=True)
transit_df.head()

In [ ]:
cancels = transit_df.groupby('FL_DATE').CANCELLED.sum()
flights = transit_df.groupby('FL_DATE').FL_DATE.count()
aFlight = (transit_df.groupby('FL_DATE').FL_DATE.count() - transit_df.groupby('FL_DATE').CANCELLED.sum())

In [ ]:
weather = transit_df.groupby('FL_DATE').WEATHER_DELAY.sum()
carrier = transit_df.groupby('FL_DATE').CARRIER_DELAY.sum()
nas = transit_df.groupby('FL_DATE').NAS_DELAY.sum()
security = transit_df.groupby('FL_DATE').SECURITY_DELAY.sum()
lateAircraft = transit_df.groupby('FL_DATE').LATE_AIRCRAFT_DELAY.sum()
delays = weather + carrier + nas + security + lateAircraft
destinations = transit_df["DEST_CITY_NAME"].unique()
len(destinations)

In [ ]:
dates = np.arange(1, 30, 1)
# dte = cancelledByDate
dte = ("1", "2","3", "4", "5", "6", "7", "8", "9", "10",
       "11", "12","13", "14", "15", "16", "17", "18", "19", "20",
       "21", "22","23", "24", "25", "26", "27", "28", "29", "30",)

In [ ]:
index = np.arange(len(dte))#how many things to plot
# plt.plot(<X AXIS VALUES HERE>, <Y AXIS VALUES HERE>, 'line type', label='label here')
# plt.plot(<X AXIS VALUES HERE>, <Y AXIS VALUES HERE>, 'line type', label='label here')
plt.figure(figsize=(20,10))
plt.plot(index, cancels, 'r-', label="Cancelled Flights")
#plt.plot(index, flights, 'b-', label="Attempted Flights")
plt.plot(index, aFlight, 'y-', label="Flown Flights")
#plt.plot(index, delays, 'y-', label="Delays")
plt.xticks(index, dte, fontsize=15)
plt.title('Flights by (date)', fontsize=15)
plt.xlabel('September 2017', fontsize=15)
plt.ylabel('Flights', fontsize=15)
plt.axvspan(8,13,color='y',alpha=0.5, label="Irma")
plt.axvspan(18,20,color='orange',alpha=0.5, label="Maria")
plt.legend(loc='best', fontsize=15)
plt.show()

In [ ]:
index = np.arange(len(dte))#how many things to plot
# plt.plot(<X AXIS VALUES HERE>, <Y AXIS VALUES HERE>, 'line type', label='label here')
# plt.plot(<X AXIS VALUES HERE>, <Y AXIS VALUES HERE>, 'line type', label='label here')
# plt.plot(index, delays, 'r-', label="Tot Delays")
plt.figure(figsize=(20,10))
plt.plot(index, carrier, '-', label="Carrier")
plt.plot(index, weather, '-', label="Weather")
plt.plot(index, nas, '-', label="NAS")
plt.plot(index, security, '-', label="Security")
plt.plot(index, lateAircraft, '-', label="Late Aircraft")
plt.xticks(index, dte, fontsize=15)
plt.xlabel('September 2017', fontsize=15)
plt.title('Delays by (date)', fontsize=15)
plt.ylabel('Delays (Minutes)', fontsize=15)
plt.axvspan(8,13,color='y',alpha=0.5, label="Irma")
plt.axvspan(18,20,color='orange',alpha=0.5, label="Maria")
plt.legend(loc='best', fontsize=15)
plt.show()

# Analysis of California

In [ ]:
# Let's group by state now and sum the number of delays.
by_origin_state = transit_df.groupby('ORIGIN_STATE_ABR')
departure_delay_counts = by_origin_state.DEP_DEL15.sum()
cancel_delay_counts = by_origin_state.CANCELLED.sum()

In [ ]:
by_dest_state = transit_df.groupby('DEST_STATE_ABR')
arrival_delay_counts = by_dest_state.ARR_DEL15.sum()

In [ ]:
# Plot, we'll put both series in a DataFrame so we can view the arrival and departure delays for each state
delay_df = pd.DataFrame([departure_delay_counts, arrival_delay_counts, cancel_delay_counts]).T

In [ ]:
delay_df.sort_values('DEP_DEL15', ascending=False).plot(kind='bar', title='Number of delayed flights by state', figsize=(20,10))
# plt.figure(figsize=(20,10))
# plt.show()

In [ ]:
cali = transit_df.loc[(transit_df["ORIGIN_STATE_ABR"] == "CA")]
cali= cali.reset_index()
cdelay = cali.groupby(["ORIGIN"]).sum()
cdelay = cdelay["DEP_DELAY"]
ccancelled = cali.groupby(["ORIGIN"]).sum()
ccancelled = ccancelled["CANCELLED"]
cdate_delay_count = cali.groupby('FL_DATE')
ca_aFlight = (cali.groupby('FL_DATE').FL_DATE.count() - cali.groupby('FL_DATE').CANCELLED.sum())
cdate_cancels = cali.groupby('FL_DATE').CANCELLED.sum()
cdate_delay_count = cali.loc[(cali["DEP_DEL15"] > 0)]
cdate_delay_count = cdate_delay_count.groupby('FL_DATE').DEP_DEL15.sum()

In [ ]:
# from pandas.io.html import read_html
# wiki_fires = pd.read_html("https://en.wikipedia.org/wiki/2017_California_wildfires")
# wildfire_df = wiki_fires[1]
# wildfire_df
# wildfire_df.columns = wildfire_df.iloc[0]
# wildfire_df = wildfire_df[1:]
# wildfire_df = wildfire_df.drop(["Notes"],axis=1)
# wildfire_df["Acres"] = wildfire_df["Acres"].astype("int64")
# wildfire_df = wildfire_df.sort_values("Acres",ascending=False)
# wildfire_df = wildfire_df.reset_index()

wildfire_sept = pd.read_csv("wildfires.csv")
wildfire_sept.head(20)

In [ ]:
plt.figure(figsize=(20,10))
airports = ["ACV", "BFL", "BUR", "FAT", "LAX", "LGB","MRY", "OAK", "ONT", "PSP" ,"RDD" ,"SAN", "SBA", "SBP", "SFO", "SJC" ,"SMF", "SNA", "STS"]
x_axis = np.arange(len(cdelay))
barlist = plt.bar(x_axis,cdelay,color="forestgreen",align="edge")
tick_locations = [value+0.4 for value in x_axis]
plt.xticks(tick_locations, airports, fontsize=15)
plt.xlim(-0.25, len(x_axis))
plt.ylim(0, max(cdelay))
plt.title("Airline Departure Delays in California During the Month of September 2017", fontsize=15)
plt.xlabel("Airport", fontsize=15)
plt.ylabel("Time of Delay (minutes)", fontsize=15)
plt.legend(loc='best', fontsize=15)
plt.show()

plt.figure(figsize=(20,10))
airports = ["ACV", "BFL", "BUR", "FAT", "LAX", "LGB","MRY", "OAK", "ONT", "PSP" ,"RDD" ,"SAN", "SBA", "SBP", "SFO", "SJC" ,"SMF", "SNA", "STS"]
x_axis = np.arange(len(ccancelled))
barlist = plt.bar(x_axis,ccancelled,color="crimson",align="edge")
tick_locations = [value+0.4 for value in x_axis]
plt.xticks(tick_locations, airports, fontsize=15)
plt.xlim(-0.25, len(x_axis))
plt.ylim(0, max(ccancelled))
plt.title("Airline Cancellations in California During the Month of September 2017", fontsize=15)
plt.xlabel("Airport", fontsize=15)
plt.ylabel("Number of Cancellations", fontsize=15)
plt.legend(loc='best', fontsize=15)
plt.show()

In [ ]:
index = np.arange(len(dte))
plt.figure(figsize=(20,10))
plt.plot(index, cdate_delay_count, 'r-', label="Delayed Flights")
plt.plot(index, cdate_cancels, "blue", label="Cancelled Flights")
plt.plot(index, ca_aFlight, 'black', label="Flown Flights")
plt.xticks(index, dte, fontsize=15)
plt.title('California Flights by (date)', fontsize=15)
plt.xlabel('September 2017', fontsize=15)
plt.axvspan(13,23,color='forestgreen',alpha=0.5, label="10 wildfires burning on this date")
plt.axvspan(26,29,color='blue',alpha=0.5, label="11 wildfires burning on this date")
plt.axvspan(9,11,color='blue',alpha=0.5)
plt.axvspan(12,13,color='blue',alpha=0.5)
plt.axvspan(23,24,color='blue',alpha=0.5)
plt.axvspan(11,12,color='yellow',alpha=0.5, label="12 wildfires burning on this date")
plt.axvspan(24,26, color='yellow',alpha=0.5,)
plt.axvspan(0,2,color='orange',alpha=0.5, label="13 wildfires burning on this date")
plt.axvspan(6,9,color='orange',alpha=0.5)
plt.axvspan(2,6,color='red',alpha=0.5, label="14 wildfires burning on this date")
plt.ylabel('Flights', fontsize=15)
plt.legend(loc='best', fontsize=15)
plt.show()

In [ ]:
#function to return conditions
def get_conditions(readDate):
    api_conditions_url = "http://api.wunderground.com/api/" + WUNDERGROUND_API_KEY + "/history_" + readDate + "/q/" + STATE +"/"+ CITY + ".json"
    try:
        f = requests.get(api_conditions_url).json()
        print(api_conditions_url)
    except:
        print("Failed to get conditions")
        return False
    if CITY == "SCE":
        return False
    else:
        return f

In [ ]:
#read csv again
csv_path = "data.csv"
weather_df = pd.read_csv(csv_path, low_memory=False)
weather_df.head()

# CREATE CHART WITH WEATHER

In [ ]:
weather_delays_df = weather_df.loc[((weather_df["WEATHER_DELAY"] > 0)) & ((weather_df["ORIGIN"]=="PHL") | (weather_df["ORIGIN"]=="DFW") | (weather_df["ORIGIN"]=="BOS") | (weather_df["ORIGIN"]=="JFK") | (weather_df["ORIGIN"]=="IAH") | (weather_df["ORIGIN"]=="DEN") | (weather_df["ORIGIN"]=="LGA") | (weather_df["ORIGIN"]=="ORD") | (weather_df["ORIGIN"]=="EWR") | (weather_df["ORIGIN"]=="SFO") | (weather_df["DEST"]=="PHL") | (weather_df["DEST"]=="DFW") | (weather_df["DEST"]=="BOS") | (weather_df["DEST"]=="JFK") | (weather_df["DEST"]=="IAH") | (weather_df["DEST"]=="DEN") | (weather_df["DEST"]=="LGA") | (weather_df["DEST"]=="ORD") | (weather_df["DEST"]=="EWR") | (weather_df["DEST"]=="SFO")) & (weather_df["DEST"]!="OMA") & (weather_df["ORIGIN"]!="OMA"),:]
weather_delays_df.reset_index(inplace=True)
weather_delays_df.head()

In [ ]:
weather_map_df = pd.DataFrame(weather_delays_df.loc[:,["YEAR", "MONTH", "DAY_OF_MONTH", "DAY_OF_WEEK",
                                          "FL_DATE","CARRIER","ORIGIN","ORIGIN_STATE_ABR","DEST","DEST_STATE_ABR","DEP_TIME","CRS_ARR_TIME","WEATHER_DELAY","CANCELLATION_CODE"]])
weather_map_df.head()

In [ ]:
#create columns for weather conditions to be pulled
weather_map_df["Conds_Description"] = "conds"
weather_map_df["Visibility"] = 0.0
weather_map_df["Pressure"] = 0.0
weather_map_df["Precipitation"] = 0.0
weather_map_df["Wind"] = 0.0
weather_map_df["Wind_Gust"] = 0.0 
weather_map_df["Fog"] = 0.0
weather_map_df["Snow"] = 0.0
weather_map_df["Temp"] = 0.0
weather_map_df["Thunder"] = 0.0
weather_map_df["Tornado"] = 0.0
weather_map_df["Rain"] = 0.0
weather_map_df["Hail"] = 0.0

weather_map_df["Dest_Conds_Description"] = "conds"
weather_map_df["Dest_Visibility"] = 0.0
weather_map_df["Dest_Pressure"] = 0.0
weather_map_df["Dest_Precipitation"] = 0.0
weather_map_df["Dest_Wind"] = 0.0
weather_map_df["Dest_Wind_Gust"] = 0.0 
weather_map_df["Dest_Fog"] = 0.0
weather_map_df["Dest_Snow"] = 0.0
weather_map_df["Dest_Temp"] = 0.0
weather_map_df["Dest_Thunder"] = 0.0
weather_map_df["Dest_Tornado"] = 0.0
weather_map_df["Dest_Rain"] = 0.0
weather_map_df["Dest_Hail"] = 0.0

In [ ]:
#wunderground keys
STARTDATE2 = ""
WUNDERGROUND_API_KEY = "aa2877982956f024"
BUCKET_KEY = "wu1"
ACCESS_KEY = "aa2877982956f024"

#loop through weather delays
weather_delays_count = len(weather_map_df)
for row in range(weather_delays_count-1):
    STATE = weather_map_df.iloc[row]["ORIGIN_STATE_ABR"]
    CITY = weather_map_df.iloc[row]["ORIGIN"]
    row_year = int(weather_map_df.iloc[row]["YEAR"])
    row_month = int(weather_map_df.iloc[row]["MONTH"])
    row_day = int(weather_map_df.iloc[row]["DAY_OF_MONTH"])
    
    STARTDATE2 = date(row_year,row_month,row_day)
    
    date_wu = []
    humidity_wu = []
    temp_wu = []
    pressure_wu = []
    precip_wu = []
    wind_speed_wu = []
    dewpt_wu = []
    wind_gust_wu = []
    wind_dir_wu = []
    vis_wu = []
    wind_chill_wu = []
    heat_index_wu = []
    conds_wu = []
    fog_wu = []
    rain_wu = []
    snow_wu = []
    hail_wu = []
    thunder_wu = []
    tornado_wu = []
    
    dest_date_wu = []
    dest_humidity_wu = []
    dest_temp_wu = []
    dest_pressure_wu = []
    dest_precip_wu = []
    dest_wind_speed_wu = []
    dest_dewpt_wu = []
    dest_wind_gust_wu = []
    dest_wind_dir_wu = []
    dest_vis_wu = []
    dest_wind_chill_wu = []
    dest_heat_index_wu = []
    dest_conds_wu = []
    dest_fog_wu = []
    dest_rain_wu = []
    dest_snow_wu = []
    dest_hail_wu = []
    dest_thunder_wu = []
    dest_tornado_wu = []

    #pull conditions for origin city on day of flight
    conditions = get_conditions(STARTDATE2.strftime("%Y%m%d"))

    if (conditions != False):
        for i in range(len(conditions['history']['observations'])):
            dateInfo = conditions['history']['observations'][i]['date']
            date_holder = dateInfo['mon']+"/"+dateInfo['mday']+"/"+dateInfo['year']+" "+dateInfo['hour']+dateInfo['min']
            pattern = '%m/%d/%Y %H%M'
            date_wu.append(date_holder)
            epoch = int(time.mktime(time.strptime(date_holder, pattern)))

            humidity_wu.append(conditions['history']['observations'][i]['hum'])
            temp_wu.append(conditions['history']['observations'][i]['tempi'])
            pressure_wu.append(conditions['history']['observations'][i]['pressurei'])
            precip_wu.append(conditions['history']['observations'][i]['precipi'])
            wind_speed_wu.append(conditions['history']['observations'][i]['wspdi'])
            dewpt_wu.append(conditions['history']['observations'][i]['dewpti'])
            wind_gust_wu.append(conditions['history']['observations'][i]['wgusti'])
            wind_dir_wu.append(conditions['history']['observations'][i]['wdird'])
            vis_wu.append(conditions['history']['observations'][i]['visi'])
            wind_chill_wu.append(conditions['history']['observations'][i]['windchilli'])
            heat_index_wu.append(conditions['history']['observations'][i]['heatindexi'])
            conds_wu.append(conditions['history']['observations'][i]['conds'])
            fog_wu.append(conditions['history']['observations'][i]['fog'])
            rain_wu.append(conditions['history']['observations'][i]['rain'])
            snow_wu.append(conditions['history']['observations'][i]['snow'])
            hail_wu.append(conditions['history']['observations'][i]['hail'])
            thunder_wu.append(conditions['history']['observations'][i]['thunder'])
            tornado_wu.append(conditions['history']['observations'][i]['tornado'])

        obs_df = pd.DataFrame({
            "Date":date_wu,
            "Temp":temp_wu,
            "Pressure":pressure_wu,
            "Precipitation":precip_wu,
            "Wind_Gust":wind_gust_wu,
            "Wind":wind_speed_wu,
            "Visibility":vis_wu,
            "Conds_Description":conds_wu,
            "Fog":fog_wu,
            "Rain":rain_wu,
            "Snow":snow_wu,
            "Hail":hail_wu,
            "Thunder":thunder_wu,
            "Tornado":tornado_wu
        })

        #find the closest observation to the scheduled flight time
        military_time = int(weather_map_df.iloc[row]["DEP_TIME"])
        if military_time < 1000:
            time_str = "0" + str(military_time)
        else:
            time_str = str(military_time)

        rcd_time = str(row_month) + "/" + str(row_day) + "/" + str(row_year) + " " + str(time_str)
        rec_time = datetime.strptime(rcd_time, '%m/%d/%Y %H%M')

        closest_time = rec_time
        min_diff = 99999999999

        converted_timestamps = []
        for raw_time in date_wu:
            converted_time = datetime.strptime(raw_time, '%m/%d/%Y %H%M')
            converted_timestamps.append(converted_time)
            diff = rec_time - converted_time
            if diff.seconds < min_diff:
                min_diff = diff.seconds
                closest_time = converted_time

        closest = closest_time.strftime("%m/%d/%Y %H%M")        

        #add conditions closest to departure time to main data frame
        observation = obs_df.loc[obs_df["Date"]==closest,:]
        observation_fog = observation["Fog"]
        observation_pressure = observation["Pressure"]
        observation_snow = observation["Snow"]
        observation_temp = observation["Temp"]
        observation_thunder = observation["Thunder"]
        observation_wind = observation["Wind"]
        observation_wind_gust = observation["Wind_Gust"]
        observation_rain = observation["Rain"]
        observation_tornado = observation["Tornado"]
        observation_visibility = observation["Visibility"]
        observation_conds_description = observation["Conds_Description"]
        observation_precipitation = observation["Precipitation"]
        observation_hail = observation["Hail"]

        weather_map_df.loc[row,"Fog"] = float(observation_fog)
        weather_map_df.loc[row,"Pressure"] = float(observation_pressure)
        weather_map_df.loc[row,"Snow"] = float(observation_snow)
        weather_map_df.loc[row,"Temp"] = float(observation_temp)
        weather_map_df.loc[row,"Thunder"] = float(observation_thunder)
        weather_map_df.loc[row,"Wind"] = float(observation_wind)
        weather_map_df.loc[row,"Wind_Gust"] = float(observation_wind_gust)
        weather_map_df.loc[row,"Rain"] = float(observation_rain)
        weather_map_df.loc[row,"Tornado"] = float(observation_tornado)
        weather_map_df.loc[row,"Visibility"] = float(observation_visibility)
        weather_map_df.loc[row,"Conds_Description"] = str(observation_conds_description)
        weather_map_df.loc[row,"Hail"] = float(observation_hail)

    #get destination conditions - same process as above
    STATE = weather_map_df.iloc[row]["DEST_STATE_ABR"]
    CITY = weather_map_df.iloc[row]["DEST"]  
             
    dest_conditions = get_conditions(STARTDATE2.strftime("%Y%m%d"))
    
    if (dest_conditions != False):
        for i in range(len(dest_conditions['history']['observations'])):
            dateInfo = dest_conditions['history']['observations'][i]['date']
            date_holder = dateInfo['mon']+"/"+dateInfo['mday']+"/"+dateInfo['year']+" "+dateInfo['hour']+dateInfo['min']
            pattern = '%m/%d/%Y %H%M'
            dest_date_wu.append(date_holder)
            epoch = int(time.mktime(time.strptime(date_holder, pattern)))

            dest_humidity_wu.append(dest_conditions['history']['observations'][i]['hum'])
            dest_temp_wu.append(dest_conditions['history']['observations'][i]['tempi'])
            dest_pressure_wu.append(dest_conditions['history']['observations'][i]['pressurei'])
            dest_precip_wu.append(dest_conditions['history']['observations'][i]['precipi'])
            dest_wind_speed_wu.append(dest_conditions['history']['observations'][i]['wspdi'])
            dest_dewpt_wu.append(dest_conditions['history']['observations'][i]['dewpti'])
            dest_wind_gust_wu.append(dest_conditions['history']['observations'][i]['wgusti'])
            dest_wind_dir_wu.append(dest_conditions['history']['observations'][i]['wdird'])
            dest_vis_wu.append(dest_conditions['history']['observations'][i]['visi'])
            dest_wind_chill_wu.append(dest_conditions['history']['observations'][i]['windchilli'])
            dest_heat_index_wu.append(dest_conditions['history']['observations'][i]['heatindexi'])
            dest_conds_wu.append(dest_conditions['history']['observations'][i]['conds'])
            dest_fog_wu.append(dest_conditions['history']['observations'][i]['fog'])
            dest_rain_wu.append(dest_conditions['history']['observations'][i]['rain'])
            dest_snow_wu.append(dest_conditions['history']['observations'][i]['snow'])
            dest_hail_wu.append(dest_conditions['history']['observations'][i]['hail'])
            dest_thunder_wu.append(dest_conditions['history']['observations'][i]['thunder'])
            dest_tornado_wu.append(dest_conditions['history']['observations'][i]['tornado'])

        dest_obs_df = pd.DataFrame({
            "Date":dest_date_wu,
            "Temp":dest_temp_wu,
            "Pressure":dest_pressure_wu,
            "Precipitation":dest_precip_wu,
            "Wind_Gust":dest_wind_gust_wu,
            "Wind":dest_wind_speed_wu,
            "Visibility":dest_vis_wu,
            "Conds_Description":dest_conds_wu,
            "Fog":dest_fog_wu,
            "Rain":dest_rain_wu,
            "Snow":dest_snow_wu,
            "Hail":dest_hail_wu,
            "Thunder":dest_thunder_wu,
            "Tornado":dest_tornado_wu
        })

        military_time = int(weather_map_df.iloc[row]["CRS_ARR_TIME"])
        if military_time < 1000:
            time_str = "0" + str(military_time)
        else:
            time_str = str(military_time)

        closest_time = rec_time
        min_diff = 99999999999

        converted_timestamps = []
        for raw_time in dest_date_wu:
            converted_time = datetime.strptime(raw_time, '%m/%d/%Y %H%M')
            converted_timestamps.append(converted_time)
            diff = rec_time - converted_time
            if diff.seconds < min_diff:
                min_diff = diff.seconds
                closest_time = converted_time

        closest = closest_time.strftime("%m/%d/%Y %H%M")        

        observation = dest_obs_df.loc[dest_obs_df["Date"]==closest,:]
        observation_fog = observation["Fog"]
        observation_pressure = observation["Pressure"]
        observation_snow = observation["Snow"]
        observation_temp = observation["Temp"]
        observation_thunder = observation["Thunder"]
        observation_wind = observation["Wind"]
        observation_wind_gust = observation["Wind_Gust"]
        observation_rain = observation["Rain"]
        observation_tornado = observation["Tornado"]
        observation_visibility = observation["Visibility"]
        observation_conds_description = observation["Conds_Description"]
        observation_precipitation = observation["Precipitation"]
        observation_hail = observation["Hail"]

        weather_map_df.loc[row,"Dest_Fog"] = float(observation_fog)
        weather_map_df.loc[row,"Dest_Pressure"] = float(observation_pressure)
        weather_map_df.loc[row,"Dest_Snow"] = float(observation_snow)
        weather_map_df.loc[row,"Dest_Temp"] = float(observation_temp)
        weather_map_df.loc[row,"Dest_Thunder"] = float(observation_thunder)
        weather_map_df.loc[row,"Dest_Wind"] = float(observation_wind)
        weather_map_df.loc[row,"Dest_Wind_Gust"] = float(observation_wind_gust)
        weather_map_df.loc[row,"Dest_Rain"] = float(observation_rain)
        weather_map_df.loc[row,"Dest_Tornado"] = float(observation_tornado)
        weather_map_df.loc[row,"Dest_Visibility"] = float(observation_visibility)
        weather_map_df.loc[row,"Dest_Conds_Description"] = str(observation_conds_description)
        weather_map_df.loc[row,"Dest_Hail"] = float(observation_hail)
        
weather_map_df

In [ ]:
#save file to csv - start next block of code based on most records received before limit reached
weather_map_df.to_csv("book2.csv", index=False, header=True)

In [ ]:
weather_map_df.head()

# PETE'S CODE

In [ ]:
### Read in CSV and elminate rows where measurements do not  make sense
weather_map_df = pd.read_csv("Book2.csv")
weather_map_df = weather_map_df[weather_map_df.Temp != 0]
weather_map_df = weather_map_df[weather_map_df.Pressure != 0]
weather_map_df.head()

In [ ]:
### Create new df with only oberservations to be analyzed
plots = pd.DataFrame(weather_map_df.loc[:,["WEATHER_DELAY","Wind","Pressure","Fog","Visibility","Temp"]])

In [ ]:
### Create scatterplots

import matplotlib.pyplot as plt
plt.figure(figsize = (12,8))
plt.scatter(plots["Wind"],plots["WEATHER_DELAY"],
            alpha = 0.75,
            edgecolor = "black")
plt.title("Flight Delay Length vs Severity of Wind")
plt.xlabel("Wind Speed [mph]")
plt.ylabel("Length of Delay [minutes]")
plt.show()

In [ ]:
plt.figure(figsize= (10,7))
plt.scatter(plots["Pressure"],plots["WEATHER_DELAY"],
            alpha = 0.75,
            edgecolor = "black")
plt.title("Flight Delay Length vs Pressure")
plt.xlabel("Pressure [Inches of Mercury]")
plt.ylabel("Length of Delay [minutes]")
plt.xlim(29.5,30.5)
pressure_floats = plots["Pressure"].astype(float).sort_values()
delay_floats = plots["WEATHER_DELAY"].astype(float)

from scipy import stats

plt.show()


In [ ]:
plt.figure(figsize=(12,8))
plt.scatter(plots["Visibility"],plots["WEATHER_DELAY"],
            alpha = 0.75,
            edgecolor = "black")
plt.title("Flight Delay Length vs Visibility")
plt.xlabel("Visibility [miles]")
plt.ylabel("Length of Delay [minutes]")
#plt.plot(x, m*x + b, '-')
plt.show()

In [ ]:
plt.figure(figsize=(12,8))
plt.scatter(plots["Temp"],plots["WEATHER_DELAY"],
            alpha = 0.75,
            edgecolor = "black")
plt.title("Flight Delay Length vs Temperature")
plt.xlabel("Temperature [F]")
plt.ylabel("Length of Delay [minutes]")
plt.show()

In [ ]:
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score
import scipy
#regr = linear_model.LinearRegression()
#regr.fit(pressure_floats, delay_floats)
from pandas.plotting import scatter_matrix
#import matplotlib.pyplot as plt
from sklearn import model_selection
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

In [ ]:
ml = pd.DataFrame(weather_map_df.loc[:,["Wind","Pressure","Visibility","Temp","Delay Bins"]])
bins = [0, 15, 30, 45, 60, 120, 180,360,100000]
labels = ["< 15","15< x <30","30,45","45,60","60,120","120,180","180,360","> 360"]
weather_map_df["Delay Bins"] = pd.cut(weather_map_df["WEATHER_DELAY"], bins=bins, labels = labels)

ml = pd.DataFrame(weather_map_df.loc[:,["Wind","Pressure","Visibility","Temp","Delay Bins"]])
ml.head()

In [ ]:
plt.figure(figsize = (15,8))

scatter_matrix(ml,figsize = (12,8))
ax1 = plt.subplot(442)
ax1.set_xlim([29.5,30.5])
plt.show()

In [ ]:
import seaborn as sns
sns.pairplot(ml, hue = "Delay Bins")
plt.show()

In [ ]:
array = ml.values
x = array[:,0:4]
y = array[:,4]
validation_size = 0.20
seed = 7
scoring = "accuracy"
x_train, x_valid, y_train, y_valid = model_selection.train_test_split(x, y, test_size=validation_size, random_state=seed)

In [ ]:
array = ml.values

x = array[:,0:4]

y = array[:,4]

validation_size = 0.20
seed = 7
scoring = "accuracy"
x_train, x_valid, y_train, y_valid = model_selection.train_test_split(x, y, test_size=validation_size, random_state=seed)

In [ ]:
# Spot Check Algorithms
models = []
models.append(('LR', LogisticRegression()))
models.append(('LDA', LinearDiscriminantAnalysis()))
models.append(('KNN', KNeighborsClassifier()))
models.append(('CART', DecisionTreeClassifier()))
models.append(('NB', GaussianNB()))
models.append(('SVM', SVC()))
# evaluate each model in turn
results = []
names = []
for name, model in models:
    kfold = model_selection.KFold(n_splits=10, random_state=seed)
    cv_results = model_selection.cross_val_score(model, x_train, y_train, cv=kfold, scoring=scoring)
    results.append(cv_results)
    names.append(name)
    perc = round(cv_results.mean()*100,2)  ## taking the mean becuse it is passing ten times
    std = round(cv_results.std(),2)
    msg = f"{name} has an accuracy percentage of {perc}% and a standard deviation of {std}"
    #msg = "%s: %f (%f)" % (name, round(cv_results.mean()*100, cv_results.std())
    print(msg)

Results
SVM, CART and KNN are our top two finishers:
SVM is "Support Vector Machines" - attempts to create a hyperplane separating classifications, and assigns test data a classification based on where in relation to the hyperplane that point lies
CART is "Decision Tree" - attempts to create a series of hierachichal decisions to lead to a classification
KNN is "k-Nearest Neighbot" - attemps to classify test data by analyzing nearest known data 
Analysis
These models are really bad...
Consistently less than 30% accuracy does not allow us to confidently classify delay ranges based on weather input at this time

In [ ]:
# Compare Algorithms
fig = plt.figure(figsize = (12,8))
#fig.suptitle('Algorithm Comparison')
plt.title("Algorithm Comparison with 8 Bins")
ax = fig.add_subplot(111)
plt.boxplot(results)
ax.set_xticklabels(names)
plt.xlabel("Algorithm")
plt.ylabel("Accuracy")
#plt.y_ticks
plt.show()

In [ ]:
# Compare Algorithms
# Make predictions on validation dataset

knn = KNeighborsClassifier()
svm = SVC()
lr = LogisticRegression()
lda = LinearDiscriminantAnalysis()
cart = DecisionTreeClassifier()
nb = GaussianNB()
tests = [knn, svm, lr, lda, cart, nb]
for test in tests:
    test.fit(x_train, y_train)
    predictions = test.predict(x_valid)
    print("Accuracy of " + str(test) + ": " + str((round(accuracy_score(y_valid, predictions)*100,2))) + "%")
    print("=============")
    print("CONFUSION MATRIX")
    print(confusion_matrix(y_valid, predictions))
    #print(classification_report(y_valid, predictions))


# DAYS OF WEEK DELAYS

In [ ]:
all_delays_df = weather_df.loc[(weather_df["DEP_DELAY"]>0),:]
all_delays_df.reset_index(inplace=True)
all_delays_df.head()

In [ ]:
weekday_delay_df = pd.DataFrame(all_delays_df.loc[:,["FL_DATE","DAY_OF_WEEK","UNIQUE_CARRIER",
                                                "ORIGIN","DEP_TIME","DEP_TIME_BLK","DEP_DELAY"]])
weekday_delay_df.head()

In [ ]:
grouped_weekday = weekday_delay_df.groupby(["DAY_OF_WEEK"])
grouped_weekday_delay = pd.DataFrame(grouped_weekday["DEP_DELAY"].mean())
grouped_weekday_delay

In [ ]:
grouped_weekday_delay.plot(kind='bar', title='Average Departure Delay by Hub Flight Type', figsize=(20,10))
plt.savefig("delaybyday.png")
plt.show()

In [ ]:
grouped_timeblk = weekday_delay_df.groupby(["DEP_TIME_BLK"])
timeblk = pd.DataFrame(grouped_timeblk["DEP_DELAY"].mean())

In [ ]:
timeblk.plot(kind='bar', title='Average Delay by Time of Day', figsize=(20,10))
plt.savefig("delaybytimeblock.png")
plt.show()

In [ ]:
# Create reference to CSV file
csv_path_hub = "cy16-commercial-service-enplanements.csv"

# Import the CSV into a pandas DataFrame
hub_df = pd.read_csv(csv_path_hub, low_memory=False)
hub_df.head()

In [ ]:
weather_hub_df = pd.DataFrame(weather_df.loc[:,["FL_DATE","CARRIER","ORIGIN","HUB_SIZE","ORIGIN_STATE_ABR","DEST","DEST_HUB_SIZE","DEST_STATE_ABR","DEP_TIME","WEATHER_DELAY","CARRIER_DELAY","NAS_DELAY","SECURITY_DELAY","CANCELLATION_CODE"]])
weather_hub_df.head()

In [ ]:
hub_weather_delays_df = weather_hub_df.loc[(((weather_hub_df["WEATHER_DELAY"] > 0) | (weather_hub_df["CARRIER_DELAY"] > 0) | (weather_hub_df["NAS_DELAY"] > 0) | (weather_hub_df["SECURITY_DELAY"] > 0)) & ((weather_hub_df["HUB_SIZE"] != "N") & (weather_hub_df["HUB_SIZE"] != "None")) & ((weather_hub_df["DEST_HUB_SIZE"] != "N") & (weather_hub_df["DEST_HUB_SIZE"] != "None")))]
hub_weather_delays_df.reset_index(inplace=True)
hub_weather_delays_df.head()

In [ ]:
hub_weather_delays_df["HUB_FLIGHT"] = hub_weather_delays_df["HUB_SIZE"] + hub_weather_delays_df["DEST_HUB_SIZE"]

In [ ]:
# Let's group by state now and sum the number of delays.
by_hub_size = hub_weather_delays_df.groupby('HUB_FLIGHT')
weather_delay_average = by_hub_size["WEATHER_DELAY"].mean()
weather_delay_average

In [ ]:
carrier_delay_average = by_hub_size["CARRIER_DELAY"].mean()
carrier_delay_average

In [ ]:
nas_delay_average = by_hub_size["NAS_DELAY"].mean()
nas_delay_average

In [ ]:
security_delay_average = by_hub_size["SECURITY_DELAY"].mean()
security_delay_average

In [ ]:
delay_df = pd.DataFrame([weather_delay_average]).T
print(delay_df)

In [ ]:
delay_df = pd.DataFrame([weather_delay_average, carrier_delay_average, nas_delay_average]).T
delay_df

In [ ]:
delay_df.plot(kind='bar', title='Average Delay by Hub Flight Type', figsize=(20,10))
plt.savefig("delaybyhub.png")
plt.show()